In [253]:
import sqlite3
import re
from bs4 import BeautifulSoup, NavigableString, Tag
import string,time
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords # it will remove stop words
exlude = string.punctuation # english punctuation list


# from textblob import TextBlob # use for spell correction TextBlob(text).correct().string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\piyus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [254]:
NOISE_PATTERNS = [
    # Original list
    "about us", "company", "we", "our",
    "number of openings", "worker type",
    "equal opportunity", "what does the future",
    "who we are", "culture", "values","motivated",
    
    # Legal & EEO
    "affirmative action", "disability", "veteran status", 
    "gender identity", "sexual orientation", "religion",
    "national origin", "reasonable accommodation", "equal employment",
    "protected veteran", "qualified individuals", "disabled veteran",
    "minorities", "women", "e-verify", "eeo employer",
    "equal opportunity employer", "eoe", "aa/eeo", "m/f/d/v",
    "without regard to race", "age", "color", "creed",
    "marital status", "pregnancy", "genetic information",
    "citizenship status", "uniformed service", "protected class",
    "ada compliant", "title vii", "adea", "federal contractor",
    "does not discriminate", "prohibits discrimination",
    "fair chance employer", "ban the box", "second chance",
    
    # Benefits & Logistics
    "benefits", "401k", "medical", "dental", "vision", 
    "compensation", "salary range", "unlimited pto", 
    "requisition", "job id", "employment type", "work authorization",
    "health insurance", "life insurance", "flexible spending",
    "hsa", "fsa", "retirement plan", "pension", "stock options",
    "equity", "rsu", "profit sharing", "bonus structure",
    "paid time off", "vacation days", "sick leave", "parental leave",
    "maternity leave", "paternity leave", "tuition reimbursement",
    "professional development", "gym membership", "wellness program",
    "employee assistance", "commuter benefits", "remote work",
    "hybrid work", "relocation assistance", "signing bonus",
    "performance bonus", "annual bonus", "quarterly bonus",
    "insurance coverage", "disability insurance", "pet insurance",
    "competitive salary", "compensation package", "total rewards",
    
    # Marketing & Narrative
    "fast-paced", "cutting edge", "innovative", "leader in",
    "proven track record", "passionate about", "thrive in",
    "come join us", "why you'll love", "our commitment",
    "dynamic environment", "collaborative team", "rockstar",
    "ninja", "guru", "wizard", "unicorn", "superstar",
    "world-class", "best-in-class", "industry leading",
    "game changer", "disruptive", "revolutionary",
    "groundbreaking", "exciting opportunity", "unique opportunity",
    "join our team", "be part of", "make an impact",
    "change the world", "transform", "reimagine",
    "wear many hats", "hit the ground running",
    "self-starter", "go-getter", "team player",
    "work hard play hard", "fun environment", "startup mentality",
    "entrepreneurial spirit", "move fast", "fail fast",
    "growth mindset", "customer obsessed", "data driven",
    "mission driven", "purpose driven", "values driven",
    
    # Company History & Background
    "founded in", "established", "headquarters", "headquartered",
    "locations", "offices worldwide", "global presence",
    "series a", "series b", "series c", "funding round", "venture backed",
    "backed by", "investors include", "valued at",
    "revenue", "customers worldwide", "users", "clients",
    "fortune 500", "fortune 1000", "inc 5000", "forbes",
    "award winning", "recognized by", "ranked", "certified",
    "fastest growing", "industry awards", "accolades",
    "publicly traded", "nasdaq", "nyse", "private company",
    "billion dollar", "million users", "market leader",
    
    # Application Instructions
    "how to apply", "application process", "submit resume",
    "cover letter", "portfolio", "work samples",
    "references", "background check", "drug test",
    "application deadline", "no agencies", "no recruiters",
    "direct applicants only", "must be authorized",
    "sponsorship not available", "will not sponsor",
    "local candidates only", "must be willing to relocate",
    "relocation required", "onsite required",
    "in office", "return to office", "apply now",
    "interested candidates", "send resume to",
    
    # # Generic Requirements Fluff
    # "bachelor's degree or equivalent", "or equivalent experience",
    # "preferred but not required", "nice to have",
    # "plus if you have", "bonus points", "would be great",
    # "ideally you", "you might be a good fit",
    # "successful candidate will", "ideal candidate",
    # "looking for someone who", "seeking a", "in search of",
    # "we need someone", "you should have", "you must have",
    # "the right person", "perfect candidate",
    
    # Diversity & Inclusion Statements
    "diversity", "inclusion", "belonging", "equity",
    "diverse workforce", "inclusive workplace", "all backgrounds",
    "underrepresented", "diverse perspectives", "committed to diversity",
    "celebrate diversity", "embrace differences",
    "inclusive culture", "safe space", "welcoming environment",
    "equal access", "accessibility", "accommodation available",
    
    # Job Posting Metadata
    "posted on", "date posted", "closing date", "open until filled",
    "position type", "employment status", "full time", "part time",
    "contract", "temporary", "permanent", "seasonal",
    "internship", "co-op", "apprenticeship",
    "entry level", "mid level", "senior level", "executive",
    "remote eligible", "location", "reports to",
    "department", "division", "team size", "job summary",
    "position summary", "role summary", "overview",
    "job description", "position description",
    
    # COVID-related
    "vaccination required", "vaccine mandate", "covid policy",
    "health and safety", "pandemic", "contactless",
    "social distancing", "mask policy","engaging","community",
    
    # Vague Responsibilities
    "other duties as assigned", "additional responsibilities",
    "ad hoc projects", "as needed", "from time to time",
    "occasionally", "may be required to", "could include",
    "miscellaneous tasks", "various duties",
    
    # Company Perks Fluff
    "free lunch", "catered meals", "snacks", "coffee",
    "beer on tap", "ping pong", "foosball", "game room",
    "standing desks", "ergonomic", "latest technology",
    "macbook", "choice of equipment", "home office stipend",
    "learning budget", "conference attendance", "book club",
    "happy hours", "team outings", "offsites", "retreats",
    "volunteer days", "charity matching", "giving back",
    "casual dress", "dress code", "flexible hours",
    
    # Buzzwords & Jargon
    "synergy", "leverage", "utilize", "optimize", "maximize",
    "streamline", "scalable", "agile", "lean", "iterative",
    "holistic", "ecosystem", "platform", "end-to-end",
    "turnkey", "plug and play", "seamless", "robust",
    "enterprise grade", "mission critical", "strategic",
    "tactical", "proactive", "reactive", "paradigm",
    "bandwidth", "circle back", "touch base", "reach out",
    "deep dive", "drill down", "unpack", "move the needle",
    "low hanging fruit", "quick win", "north star",
    "pivoting", "disrupt", "empower", "enable",
    
    # Disclaimers
    "subject to change", "at the discretion of", "reserves the right",
    "not all inclusive", "representative only", "may vary",
    "this is not a contract", "employment at will",
    "confidential information", "proprietary", "trade secrets",
    "without notice", "subject to approval",
    
    # Contact & Social
    "follow us on", "connect with us", "social media","growing","grow"
    "linkedin", "twitter", "facebook", "instagram",
    "website", "learn more at", "visit us",
    "contact", "email", "phone", "address",
    "careers page", "jobs site",
    
    # Experience level fluff  
    "years of experience", "or more years", "minimum of",
    "at least", "prefer", "strongly prefer", "highly desired",
    "required experience", "background required",
    
    # Soft skills fluff (generic)
    "excellent communication", "strong communication",""
    "written and verbal", "interpersonal skills",
    "attention to detail", "multitask", "prioritize",
    "time management", "organizational skills",
    "critical thinking", "analytical",
    "work independently", "self-motivated", "driven",
    "results oriented", "goal oriented", "deadline driven",
    "positive attitude", "can-do attitude", "energetic",
    "flexible", "adaptable", "resilient","organisation",
    
    # # Section Headers
    # "job summary", "position summary", "about the role",
    # "what you'll do", "responsibilities include",
    # "requirements", "qualifications", "what we're looking for",
    # "about you", "you have", "what you bring",
    # "what we offer", "why join us", "perks and benefits",
    # "compensation and benefits", "the role", "the position",
    # "job details", "position details", "role details"
]

In [255]:
SKILL_HINTS = [
    # Original list
    "experience with", "knowledge of", "proficient in",
    "hands-on experience in", "nice to have", "years of experience",
    "good knowledge", "strong knowledge",

    # Proficiency & Expertise
    "solid understanding of", "background in", "familiarity with",
    "expert in", "demonstrated ability", "competency in",
    "deep understanding", "advanced knowledge", "skilled at",
    "fluency in", "proven ability to", "practices",
    "working knowledge", "comprehensive knowledge", "thorough understanding",
    "mastery of", "expertise in", "specialization in",
    "well-versed in", "conversant with", "acquainted with",
    
    # Preferred & Bonus Skills
    "preferred", "plus if", "highly desired", "a plus",
    "not required but", "advantage", "ideally", "bonus",
    "considered a plus", "beneficial", "would be nice",
    "strong preference for", "preferably with", "desirable",
    "sought after", "valued", "appreciated",
    
    # Action & Contextual Phrases
    "working with", "exposure to", "comfortable using",
    "utilizing", "background with", "proficient with",
    "adept in", "understanding of", "passion for",
    "at least", "minimum of", "up to", "years in",
    "capable of", "able to work with", "skilled in",
    "trained in", "certified in", "versed in",
    "practiced in", "experienced in using", "accustomed to",
    
    # Requirements Language
    "must have", "should have", "required to have",
    "need to have", "expected to have", "necessary to have",
    "essential", "critical", "mandatory", "prerequisite",
    "required skill", "core competency", "key skill",
    
    # Educational Context
    "degree in", "major in", "concentration in",
    "coursework in", "training in", "certification in",
    "studied", "learned", "academic background in",
    
    # Time-based Experience
    "minimum", "at least", "more than", "over",
    "years of", "months of", "experience in",
    "recent experience", "current experience",
    "prior experience", "previous experience",
    "demonstrated history", "track record with",
    
    # Application Context
    "using", "applying", "implementing", "building with",
    "developing with", "coding in", "programming in",
    "working within", "operating within", "employed in",
    
    # Level Indicators
    "beginner", "intermediate", "advanced", "senior level",
    "junior level", "entry level", "professional level",
    "production level", "enterprise level",
    
    # Soft Skill Indicators
    "ability to", "capable of", "capacity for",
    "aptitude for", "talent for", "skilled at",
    "comfortable with", "confident in",
    
    # Tech-specific Contexts
    "stack includes", "tech stack", "toolset includes",
    "frameworks such as", "technologies like",
    "platforms including", "systems such as",
    "tools like", "environments including"
]

In [256]:
TECH_KEYWORDS = [
    # Original
    "python", "java", "c#", "sql", "csharp", ".net", "linux", "cassandra", "net"
    "aws", "docker", "kubernetes", "html", "javascript", "nosql", "erp",

    # Programming Languages
    "golang", "rust", "typescript", "javascript", "ruby", "php", "swift", 
    "kotlin", "scala", "c++", "cpp", "objective-c", "perl", "bash", 
    "powershell", "shell", "ruby", "matlab", "sas", "dart", "elixir", 
    "clojure", "haskell", "f#", "fsharp", "groovy", "lua", "julia",
    "vb.net", "vba", "assembly", "cobol", "fortran",

    # Frontend Frameworks & Libraries
    "react", "reactjs", "nextjs", "next.js", "angular", "angularjs",
    "vuejs", "vue.js", "vue", "svelte", "sveltekit", "ember", "backbone",
    "jquery", "tailwind", "tailwindcss", "bootstrap", "material-ui", "mui",
    "sass", "scss", "less", "styled-components", "css3", "html5",
    "webpack", "vite", "parcel", "rollup", "gulp", "grunt",
    "redux", "mobx", "recoil", "zustand", "rxjs",
    
    # Backend Frameworks
    "django", "flask", "fastapi", "spring", "spring boot", "node.js", 
    "nodejs", "express", "expressjs", "koa", "hapi", "nestjs",
    "laravel", "symfony", "rails", "ruby on rails", "sinatra",
    "asp.net", "asp.net core", ".net core", ".net framework",
    "entity framework", "ef core", "wcf", "wpf", "winforms",
    "servlet", "jsp", "struts", "hibernate", "play framework",
    
    # API & Architecture
    "graphql", "rest", "rest api", "restful", "soap", "grpc",
    "microservices", "api gateway", "swagger", "openapi",
    "json", "xml", "protobuf", "thrift", "avro",
    "webhook", "websocket", "oauth", "jwt", "saml",
    
    # Cloud Platforms & Services
    "azure", "gcp", "google cloud", "google cloud platform",
    "aws", "amazon web services", "ec2", "s3", "rds", "lambda",
    "cloudformation", "cloudfront", "route53", "elasticbeanstalk",
    "dynamodb", "sqs", "sns", "kinesis", "emr", "redshift",
    "azure devops", "azure functions", "azure storage",
    "cloud run", "cloud functions", "app engine", "compute engine",
    "cloud storage", "cloud sql", "firebase", "heroku",
    "digitalocean", "linode", "vultr", "cloudflare",
    
    # DevOps & CI/CD
    "terraform", "terragrunt", "ansible", "puppet", "chef", "salt",
    "jenkins", "gitlab ci", "github actions", "circleci", "travis ci",
    "bamboo", "teamcity", "azure pipelines", "argo cd", "flux",
    "ci/cd", "continuous integration", "continuous deployment",
    "docker", "docker compose", "podman", "containerd",
    "kubernetes", "k8s", "helm", "kustomize", "istio", "linkerd",
    "prometheus", "grafana", "datadog", "new relic", "splunk",
    "elk", "elasticsearch", "logstash", "kibana", "fluentd",
    
    # Databases - SQL
    "sql",
    "postgresql", "postgres", "mysql", "mariadb", "oracle",
    "sql server", "mssql", "t-sql", "pl/sql", "sqlite",
    "db2", "sybase", "informix", "cockroachdb", "aurora",
    
    # Databases - 
    "nosql",
    "mongodb", "mongo", "couchdb", "couchbase", "cassandra",
    "redis", "memcached", "dynamodb", "neo4j", "arangodb",
    "orientdb", "riak", "hbase", "accumulo",
    
    # Data Engineering & Big Data
    "spark", "apache spark", "pyspark", "hadoop", "hdfs",
    "hive", "pig", "sqoop", "flume", "nifi", "airflow",
    "kafka", "apache kafka", "pulsar", "rabbitmq", "activemq",
    "flink", "storm", "beam", "databricks", "snowflake",
    "bigquery", "redshift", "athena", "presto", "trino",
    "dbt", "looker", "tableau", "power bi", "powerbi", "qlik",
    
    # Machine Learning & AI
    "tensorflow", "pytorch", "keras", "scikit-learn", "sklearn",
    "pandas", "numpy", "scipy", "matplotlib", "seaborn",
    "jupyter", "notebook", "mlflow", "kubeflow", "sagemaker",
    "xgboost", "lightgbm", "catboost", "hugging face",
    "transformers", "bert", "gpt", "llm", "nlp", "cv","computer vision",
    "opencv", "yolo", "detectron", "spacy", "nltk","statistics",
    
    # Mobile Development
    "android", "ios", "react native", "flutter", "xamarin",
    "cordova", "phonegap", "ionic", "swift", "swiftui",
    "objective-c", "kotlin", "java android", "jetpack compose",
    
    # Testing & Quality
    "selenium", "cypress", "playwright", "puppeteer", "testcafe",
    "jest", "mocha", "jasmine", "junit", "testng", "pytest",
    "unittest", "rspec", "cucumber", "behave", "postman",
    "jmeter", "gatling", "locust", "k6", "sonarqube",
    "unit testing", "integration testing", "e2e testing",
    
    # Version Control & Collaboration
    "git", "github", "gitlab", "bitbucket", "svn", "mercurial",
    "jira", "confluence", "trello", "asana", "monday",
    "slack", "teams", "zoom", "notion",
    
    # Operating Systems & Infrastructure
    "linux", "unix", "ubuntu", "centos", "rhel", "debian",
    "windows", "windows server", "macos", "freebsd",
    "nginx", "apache", "iis", "haproxy", "envoy", "traefik",
    "loadbalancer", "cdn", "cloudflare", "akamai",
    
    # Security
    "owasp", "penetration testing", "vulnerability assessment",
    "siem", "waf", "ids", "ips", "ssl", "tls", "pki",
    "vault", "secrets management", "kms", "encryption",
    
    # Methodologies & Practices
    "agile", "scrum", "kanban", "waterfall", "devops",
    "sre", "tdd", "bdd", "ddd", "clean code", "solid",
    "design patterns", "microservices", "monolith",
    "event-driven", "cqrs", "saga pattern",
    
    # ERP & Business Software
    "sap", "oracle erp", "netsuite", "microsoft dynamics",
    "dynamics 365", "nav", "microsoft nav", "navision",
    "salesforce", "crm", "workday", "servicenow",
    "peoplesoft", "jd edwards", "epicor",
    
    # Web Servers & Protocols
    "http", "https", "tcp/ip", "udp", "dns", "dhcp",
    "ftp", "sftp", "ssh", "smtp", "imap", "pop3",
    
    # Markup & Data Formats
    "html", "html5", "css", "css3", "xml", "json", "yaml",
    "toml", "markdown", "latex", "svg",
    
    # Other Technologies
    "blockchain", "ethereum", "solidity", "web3",
    "iot", "mqtt", "zigbee", "raspberry pi",
    "arduino", "embedded systems", "rtos",
    "unity", "unreal engine", "godot", "game development",
    "opengl", "vulkan", "directx", "webgl", "three.js",
    "d3.js", "chart.js", "plotly", "highcharts"
]

In [257]:
RESP_VERBS = {
    # Original
    "develop", "design", "implement", "create", "maintain", 
    "manage", "analyze", "evaluate", "deliver", "provide", 
    "support", "coordinate", "participate", "plan", "write", "observe",

    # Construction & Engineering
    "architect", "build", "code", "deploy", "debug", "engineer", 
    "integrate", "configure", "automate", "refactor", "optimize", 
    "prototype", "scale", "test", "validate", "script",
    "compile", "construct", "assemble", "synthesize", "craft",
    "program", "migrate", "upgrade", "patch", "release",
    
    # Strategy & Leadership
    "lead", "mentor", "supervise", "oversee", "spearhead", "drive", 
    "define", "roadmap", "prioritize", "influence", "guide", 
    "champion", "transform", "empower", "delegate", "direct",
    "orchestrate", "command", "govern", "steer", "pilot",
    "establish", "initiate", "launch", "pioneer",
    
    # Collaboration & Liaison
    "collaborate", "partner", "liaise", "present", "facilitate", 
    "consult", "advise", "train", "onboard", "align", "brief",
    "communicate", "engage", "interface", "network", "connect",
    "coordinate with", "work with", "team up", "cooperate",
    "sync", "share", "disseminate", "educate", "coach",
    
    # Operational & Analytical
    "monitor", "troubleshoot", "audit", "investigate", "resolve", 
    "refine", "enhance", "streamline", "modernize", "assess", 
    "forecast", "interpret", "measure", "track", "document",
    "diagnose", "identify", "detect", "discover", "uncover",
    "examine", "inspect", "review", "survey", "study",
    "quantify", "calculate", "compute", "estimate", "project",
    
    # Maintenance & Support
    "fix", "repair", "update", "maintain", "service",
    "sustain", "preserve", "uphold", "secure", "protect",
    "backup", "restore", "recover", "salvage",
    
    # Communication & Documentation
    "report", "document", "record", "log", "catalog",
    "publish", "communicate", "articulate", "explain",
    "describe", "specify", "detail", "outline", "draft",
    "prepare", "compile", "summarize", "synthesize",
    
    # Process & Workflow
    "execute", "perform", "conduct", "carry out", "run",
    "operate", "handle", "process", "administer", "control",
    "regulate", "standardize", "normalize", "systematize",
    "organize", "structure", "arrange", "order",
    
    # Improvement & Optimization
    "improve", "enhance", "optimize", "increase", "boost",
    "accelerate", "expedite", "advance", "elevate", "maximize",
    "minimize", "reduce", "decrease", "eliminate", "mitigate",
    "consolidate", "simplify", "rationalize",
    
    # Research & Innovation
    "research", "explore", "experiment", "investigate", "innovate",
    "ideate", "conceptualize", "brainstorm", "hypothesize",
    "discover", "invent", "formulate", "devise", "originate",
    
    # Quality & Compliance
    "ensure", "verify", "confirm", "certify", "approve",
    "enforce", "comply", "adhere", "follow", "meet",
    "satisfy", "fulfill", "achieve", "attain", "accomplish",
    
    # Data & Analytics
    "gather", "collect", "aggregate", "consolidate",
    "parse", "extract", "transform", "load", "etl",
    "visualize", "model", "simulate", "predict",
    "correlate", "cluster", "classify", "segment",
    
    # Deployment & Release
    "deploy", "release", "rollout", "ship", "deliver",
    "distribute", "install", "provision", "allocate",
    "stage", "promote", "push", "publish",
    
    # Security & Risk
    "secure", "protect", "safeguard", "defend", "shield",
    "encrypt", "authenticate", "authorize", "validate",
    "sanitize", "scan", "penetration test", "harden",
    
    # Acquisition & Procurement
    "acquire", "obtain", "procure", "source", "purchase",
    "negotiate", "contract", "vendor manage",
    
    # Review & Approval
    "review", "approve", "sign-off", "authorize", "endorse",
    "critique", "evaluate", "judge", "rate", "assess"
}

In [258]:
import html
def clean_jd(text:str)->str:
    # converting html into clean point text
    soup = BeautifulSoup(text,"html.parser")
     
    #removing script/style 
    for tag in soup(["script","style"]):
        tag.decompose()
    
    # getting raw text
    text = soup.get_text(separator=" ")

    # decoding html entities
    text = html.unescape(text)

    # removing non-bbreaking spaces
    text = text.replace("\xa0"," ")

    # remove bullets, tabs
    text = re.sub(r"[•\t]"," ",text)
    text = re.sub(r"\s+"," ",text).strip()
    text = text.lower()
    return text
    

In [259]:
# from bs4 import NavigableString, Tag
# import html, re

def normalize_for_dedup(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9 ]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_sentences_from_html(html_text: str):
    soup = BeautifulSoup(html_text, "html.parser")

    for tag in soup(["script", "style"]):
        tag.decompose()

    sentences = []

    def process_block(block):
        current = []

        for node in block.descendants:
            if isinstance(node, NavigableString):
                text = html.unescape(str(node))
                text = text.replace("\xa0", " ")
                text = re.sub(r"[•\t]", " ", text)
                current.append(text)

            elif isinstance(node, Tag) and node.name == "br":
                sent = "".join(current).strip()
                if sent:
                    sentences.append(sent)
                current = []

        sent = "".join(current).strip()
        if sent:
            sentences.append(sent)

    # process p & li
    for block in soup.find_all(["p", "li"]):
        process_block(block)

    # process div only if it doesn't contain p or li
    for block in soup.find_all("div"):
        if not block.find(["p", "li"]):
            process_block(block)

    # ---- DEDUPLICATION ----
    seen = set()
    final_sentences = []
    
    for s in sentences:
        s = re.sub(r"\s+", " ", s).strip().lower()
        if len(s) < 3:
            continue

        key = normalize_for_dedup(s)
        if key not in seen:
            seen.add(key)
            # s = re.split(r"[.;,<>]",s)
            final_sentences.extend(
                part.strip()
                for part in re.split(r"[.<>;]",s)
                if len(part.strip())>=3
            )

    return final_sentences


In [260]:
def split_sentences(clean_text: str):
    """
    Splits cleaned text into meaningful sentences
    """

    # Split on . ; line breaks
    sentences = re.split(r"[.;,<>]", clean_text)

    # Clean & filter
    sentences = [
        s.strip() for s in sentences
        if len(s.strip()) > 3
    ]

    return sentences


In [261]:
# description="<p style=\"text-align:left\"><u><b>Job Summary:</b></u></p>Developer NAV (Navision)  <br> <br>    The Developer NAV is responsible for a range of development tasks in projects associated with design proposals, testing, feasibility analysis, modeling, sample building etc. to develop and implement new methods, designs and products. <br> \t<br>          He/she participates in integrated project teams. <br><br>          Writes technical specifications and analyzes / interprets test results.  <br><br>Responsibilities in detail: <br> <br>•\t     Creates detailed technical specifications, requirements feasibility checks and time estimation<br>•\t       Develops complex functionalities  <br>•\t  Creates guidelines concerning development, design and layers  <br>•\t  Observes changes and future planning in Dynamics NAV technical environment  <br>•\t  Analyses and takes decisions regarding reintegration of important partner developments  <br>•\t  Evaluates third party products for possible integration  <br>•\t  Creates prototypes for presentations and further decisions / tests  <br>•\t  Delivering of internal trainings needed for Know-How-Transfer  <br>•\t  Provides second level product support for internal departments  <br>•\t  Is involved in product roadmap and release planning  <br>•\t  Delivers detailed planning and update information to project managers  <br>•\t  Is able to provide project management of certain projects in design responsibility  <br>•\t  Is able to assume responsibility for the involvement / co-ordination of further developers  <br>•\t Develops and maintains good relationships with partners, OEMs and important customers in    case of technical requirements<p style=\"text-align:inherit\"></p><p style=\"text-align:inherit\"></p><p style=\"text-align:left\"><u><b>Job Description:</b></u></p><p>Skills and Specifications:</p><p></p><ul><li></li></ul><div><p></p></div><p> Well-founded IT education or equal qualification  &lt; &gt;</p><div><p></p></div><p> 5 - 7 years of professional experience in software development  &lt; &gt;</p><div><p></p></div><p> High level technology know-how in .NET, C#, SQL  &lt; &gt;</p><div><p></p></div><p> Experience in software development based on Microsoft ERP platforms, preferably Microsoft NAV   &lt; &gt;</p><div><p></p></div><p> Proven practical experience with Client-Server programming technologies  &lt; &gt;Knowledge of AJAX, ASP.NET, WPF, WCF, Silverlight, JavaScipt or flash or HTML5, ADO.NET, VB,                      LINQ is an advantage </p><div><p></p></div><p> Ability to solve complex tasks  &lt; &gt;</p><div><p></p></div><p> Flexibility to adapt to shifting priorities with ease  &lt; &gt;</p><div><p></p></div><p> Solution orientation, strong organizing and keeping track ability  &lt; &gt;</p><div><p></p></div><p> Abstract thinking and analytical ability to decompose complex problems  &lt; &gt;</p><div><p></p></div><p> Ability to work very independently and constructively in a team  &lt; &gt;</p><div><p></p></div><p> Sound spoken and written English; additional foreign language is beneficial  </p><p></p><p></p><p style=\"text-align:inherit\"></p><p style=\"text-align:inherit\"></p><p style=\"text-align:left\"><u><b>Worker Type:</b></u></p>Regular<p></p><p></p><p style=\"text-align:left\"><b><u>Number of Openings Available:</u> &nbsp; &nbsp;</b></p>1"
description="<p>For more than 40 years, Accelya has been the industry’s partner for change, simplifying airline financial and commercial processes and empowering the air transport community to take better control of the future. Whether partnering with IATA on industry-wide initiatives or enabling digital transformation to simplify airline processes, Accelya drives the airline industry forward and proudly puts control back in the hands of airlines so they can move further, faster.<span> </span></p><p></p><p><span><span>Good knowledge on UNIX/Linux commands and Shell scripting Knowledge</span></span></p><p><span> </span></p><p><span>Min L2 Cassandra Support</span></p><p><span> </span></p><p><span><i>Nice to have options to maybe include</i>: ITIL</span></p><p><span> </span></p><p><span>Security best practises </span></p><p><span> </span></p><p><span>Sharding for scalability</span></p><p><span> </span></p><p><span>Knowledge and experience with Clustering </span></p><p><span> </span></p><p><span>Knowledge and experience with Cloud technologies</span></p><p><span> </span></p><p><span>OS Performance tuning for NoSQL</span></p><p><span> </span></p><p><span><i>Nice to have</i>: Able to complete documentation through Confluence pages</span></p><p><span> </span></p><p><span>Number of years experience 7&#43; years -- depending on budget and skill level we are looking for.</span></p><p><span> </span></p><p><span>Knowledge in installation/upgrades/migration/backup and restore/data recovery, performance tuning, Cassandra schema design, and experience in loading data into Cassandra clusters.</span></p><p><span>Very good english </span></p><h3><span> </span></h3><p></p><p></p><p>What does the future of the air transport industry look like to you? Whether you’re an industry veteran or someone with experience from other industries, we want to make your ambitions a reality!</p>"
# description = "<p style=\"text-align:left\"><span><b><b>Job Description:</b></b></span></p><p><b>Company description:</b></p><div> <p><span>Airbus pioneers sustainable aerospace for a safe and united world. The Company constantly innovates to provide efficient and technologically-advanced solutions in aerospace, defense, and connected services. In commercial aircraft, Airbus offers modern and fuel-efficient airliners and associated services. Airbus is also a European leader in defense and security and one of the world&#39;s leading space businesses. In helicopters, Airbus provides the most efficient civil and military rotorcraft solutions and services worldwide.</span></p> <p><span>Our people work with passion and determination to make the world a more connected, safer and smarter place. Taking pride in our work, we draw on each other&#39;s expertise and experience to achieve excellence. Our diversity and teamwork culture propel us to accomplish the extraordinary - on the ground, in the sky and in space.</span></p> <p><span>Founded in 2021, the Airbus Global Business Services (AGBS) unit will be contributing to Airbus’ global success and play a vital role in Airbus’ transformation journey in  standardization, harmonization, digitalisation and automation of end-to-end processes (e.g.Procure to Pay) of all overhead functions. Our target is that siloed ways of working / processes will be a relic of the past within the AGBS. To drive such cultural change  we are looking for candidates with pioneering spirit that make data-driven decisions and take responsibility for the best outcome of their internal customers. All while embracing a spirit of problem-solving and living the Airbus values: Customer focus, integrity, respect, creativity, reliability and teamwork.</span></p> <p><span>Our purpose defines why we exist: ‘We pioneer sustainable aerospace for a safe and united world’.</span></p></div><p></p><p> </p><div><p><b><b>Responsibilities - your mission</b></b></p> <p><span>We are the central Advanced Analytics and Artificial Intelligence team in Airbus for commercial aircraft and we deliver business impact through AI products and services across the entire value chain all the way from selling, designing, procuring, manufacturing to servicing aircraft. As such, we are responsible for solving all aspects and challenges of the machine learning (ML) product lifecycle, including problem exploration, solution development and deployment into cloud native environments at scale. </span><br /><br /><span>As part of our team you will work, learn and grow among an international and diverse group of top talent data scientists, software engineers, product managers and many more based across Europe and India. With truly vast amounts of data from all over the Airbus ecosystem at our disposal paired with an unparalleled product complexity, we are exposed to unique and exciting challenges, working at the forefront of AI in the aviation industry. </span></p> <p><b><b>Key responsibilities for this role include but are not limited to:</b></b></p><ul><li><p><span>Partner with business functions on missions to absorb domain expertise and priorities, build analytical insights and translate business challenges into AI related opportunities that you will tackle with our wider team</span></p></li><li><p><span>Carry out hands-on, end-to-end AI / ML model development from early ideation, exploration and feasibility studies on to design, development, deployment as well as monitoring of scaled AI services </span></p></li><li><p><span>Support our AI capability teams in defining and delivering on AI capability roadmaps (built around 5 technology pillars: computer vision, knowledge extraction, Time-series anomaly detection, decision making and hybrid modeling) that target reusable services and components which are consumed by a wide range of IT products</span></p></li><li><p><span>Representing the central AI organization, evangelize and establish best practices and guidance on delivering and running AI solutions in the rapidly growing Airbus Lisbon office </span></p></li><li><p><span>Engaging in the global AI and analytics community as well as growing the local community footprint, through holding project showcases, sharing expertise as well as best practices</span></p></li></ul> <p></p><p><b>Requirements - our ideal candidate will have</b></p> <ul><li><p><span>3&#43; years industry experience in data science, AI or a similar field with a track record of end-to-end model development and handling large data-sets</span></p></li><li><p><span>Strong, fundamental understanding of common ML algorithms</span></p></li><li><p><span>Bachelor’s, Master’s or PhD degree in Computer Science, Data Science, Statistics, Engineering, Mathematics or a related quantitative discipline</span></p></li><li><p><span>Deepened knowledge in at least one of the following fields: NLP,  Computer Vision, Time Series Analysis</span></p></li><li><p><span>High Proficiency in Python including relevant open source libraries such as Pandas, Scikit-Learn, Tensorflow and PyTorch</span></p></li><li><p><span>Fluent in written and spoken English</span></p></li><li><p><span>Experience in customer facing roles focused on business impact and managing expectations</span></p></li><li><p><span>Experience in process analysis and process optimization including methodologies such as value stream mapping</span></p></li><li><p><span>Development and deployment of models in cloud environments like AWS, GCP or SAP</span></p></li><li><p><span>Agile and product-oriented development</span></p></li></ul> <p><b><b>Benefits - what’s in it for you</b></b></p><ul><li><p><span>Diverse career opportunities within Airbus European core countries or in other regions around the world.  </span></p></li><li><p><span>A hybrid working model, allowing you to combine onsite and offsite work. </span></p></li><li><p><span>A modern office at Parque das Nacoes, well connected to public transportation.</span></p></li><li><p><span>A motivated and fun crew to grow and build and shape the GBS together.</span></p></li><li><p><span>An intense and exciting onboarding experience.</span></p></li></ul></div><p></p><p></p><p></p><p style=\"text-align:inherit\"></p><p style=\"text-align:left\">This job requires an awareness of any potential compliance risks and a commitment to act with integrity, as the foundation for the Company’s success, reputation and sustainable growth.</p><p style=\"text-align:inherit\"></p><p style=\"text-align:left\"><span><b><b><b><b>Company:</b></b></b></b></span></p>Airbus Portugal SA<p></p><p><b>Employment Type:</b></p>Permanent<p style=\"text-align:inherit\">-------</p><p style=\"text-align:inherit\"></p><p style=\"text-align:left\"><b>Experience Level:</b></p>Entry Level<p style=\"text-align:inherit\"></p><p style=\"text-align:left\"><b>Job Family:</b></p>Digital &lt;JF-IM-DI&gt;<p style=\"text-align:inherit\"></p><p style=\"text-align:left\">By submitting your CV or application you are consenting to Airbus using and storing information about you for monitoring purposes relating to your application or future employment. This information will only be used by Airbus.<br />Airbus is committed to achieving workforce diversity and creating an inclusive working environment. We welcome all applications irrespective of social and cultural background, age, gender, disability, sexual orientation or religious belief.</p><p style=\"text-align:left\"><span>Airbus is, and always has been, committed to equal opportunities for all. As such, we will never ask for any type of monetary exchange in the frame of a recruitment process. Any impersonation of Airbus to do so should be reported to </span><span><a href=\"mailto:emsom&#64;airbus.com\" target=\"_blank\">emsom&#64;airbus.com</a>.</span></p><p style=\"text-align:inherit\"></p><p style=\"text-align:left\"><span>At Airbus, we support you to work, connect and collaborate more easily and flexibly. Wherever possible, we foster flexible working arrangements to stimulate innovative thinking.</span></p>"
clean_text= clean_jd(description)
sentences = split_sentences(clean_text)

# print("CLEAN TEXT:\n")
# print(clean_text[:800], "...")  # preview

print("\nSENTENCES:\n")
for i, s in enumerate(sentences, 1):
    print(f"{i}. {s}")


SENTENCES:

1. for more than 40 years
2. accelya has been the industry’s partner for change
3. simplifying airline financial and commercial processes and empowering the air transport community to take better control of the future
4. whether partnering with iata on industry-wide initiatives or enabling digital transformation to simplify airline processes
5. accelya drives the airline industry forward and proudly puts control back in the hands of airlines so they can move further
6. faster
7. good knowledge on unix/linux commands and shell scripting knowledge min l2 cassandra support nice to have options to maybe include : itil security best practises sharding for scalability knowledge and experience with clustering knowledge and experience with cloud technologies os performance tuning for nosql nice to have : able to complete documentation through confluence pages number of years experience 7+ years -- depending on budget and skill level we are looking for
8. knowledge in installation/

In [262]:
sentences = extract_sentences_from_html(description)


In [263]:
def is_noise(sentence):
    s = normalize_for_dedup(sentence)
    for pattern in NOISE_PATTERNS:
        if re.search(f"{pattern} ", s):
            return True
    return False


In [264]:
def extract_skills(sentence):
    s = normalize_for_dedup(sentence)
    skills = set()

    # explicit tech keywords
    for tech in TECH_KEYWORDS:
        if tech.lower() in s:
            skills.add(tech)

    return list(skills)


In [265]:
sentences

['for more than 40 years, accelya has been the industry’s partner for change, simplifying airline financial and commercial processes and empowering the air transport community to take better control of the future',
 'whether partnering with iata on industry-wide initiatives or enabling digital transformation to simplify airline processes, accelya drives the airline industry forward and proudly puts control back in the hands of airlines so they can move further, faster',
 'good knowledge on unix/linux commands and shell scripting knowledge',
 'min l2 cassandra support',
 'nice to have options to maybe include: itil',
 'security best practises',
 'sharding for scalability',
 'knowledge and experience with clustering',
 'knowledge and experience with cloud technologies',
 'os performance tuning for nosql',
 'nice to have: able to complete documentation through confluence pages',
 'number of years experience 7+ years -- depending on budget and skill level we are looking for',
 'knowledge i

In [266]:
def extract_responsibility(sentence):
    s = sentence.lower()
    for verb in RESP_VERBS:
        if s.startswith(verb) or f" {verb} " in s:
            return True
    return False

In [267]:
import json
def get_skills_and_responsibility(sentences):
    extracted_skills = []
    temp_responsibility = []
    for i,s in enumerate(sentences,1):
        if extract_responsibility(s) : temp_responsibility.append(s)
        skills = extract_skills(s)
        extracted_skills.extend(skills)
    responsibility = []
    for s in temp_responsibility:
        if not is_noise(s): responsibility.append(s)
    # print(f"Before clearing noise")
    # print(temp_responsibility)
    # print(f"After removing noise")
    # responsibility
    data = {
        "skills": [skills],
        "responsibility":[responsibility],
    }
    return json.dumps(data)
